In [ ]:
import numpy as np
import pandas as pd

from scipy.signal import iirfilter, lfilter, periodogram, stft, istft
from scipy.spatial.distance import seuclidean
from scipy.stats import chi2

import matplotlib.pyplot as plt
%matplotlib notebook
figsize = [9.5, 5]

In [ ]:
files = !ls data/*.fthr | grep 29.5_ANALOGMAX-0
file = files[-292]
file

In [ ]:
df = pd.read_feather(file) 
df = df[df.time_ms < np.round(df.time_ms.values[-1] / 1000) * 1000]
df.shape

In [ ]:
def getParams(fs, alphaMax=2000, dfreq=500, fMin=5e+4):
    # alphaMax - max modulation frequency
    # dfreq - carrier frequency resolution
    # fMin - min carrier frequency

    # STFT windows' hop
    R = int(np.floor(fs / (2 * alphaMax))) #shift of the stft
    # STFT window length
    Nw = int(fs / dfreq)
    # number of STFT windows
    # M = int((x.size - Nw) / R + 1)

    # hannind window
    w = np.hanning(Nw)
    # Dirichlet kernel parameter
    P = int(np.round((Nw - 1) / (2 * R)))
    # Dirichlet kernel
    D = np.sum(
        [np.exp(2 * np.pi * 1j * p *(np.arange(Nw) - Nw / 2) / Nw) for p in np.arange(- P, P + 1)], 
        axis=0
    )
    D = D.real
    return fMin, R, Nw, w, D

In [ ]:
def getCS(x, fs, approxCoh=True, normalization=False, alphaMax=2000, dfreq=500): 
    # An implementation of the spectral correlation and spectral coherence according to 
    # Borghesani, P., and J. Antoni. "A faster algorithm for the calculation of the fast spectral correlation." 
    # Mechanical Systems and Signal Processing 111 (2018): 113-118.

    fMin, R, Nw, w, D = getParams(fs, alphaMax=alphaMax, dfreq=dfreq)

    # STFT with with Hanning window and with Hanning multiplied by Dirichlet kernel
    X_w = stft(x, fs=fs, window=w, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)[-1]
    f, t, X_w_d = stft(x, fs=fs, window=w * D, nperseg=Nw, noverlap=Nw - R, nfft=Nw, return_onesided=True)

    if approxCoh:
        # here I save some computation time by removing the frequencies below fMin.
        X_w = X_w[f >= fMin, :-1]
        X_w_d = X_w_d[f >= fMin, :-1]

    # Cyclcic Spectrum
    CS = np.fft.fft(np.conjugate(X_w) * X_w_d, axis=1).T
    # Modulation frequency
    alpha = np.fft.fftfreq(X_w_d.shape[1], R / fs)
    pistiveAlphaCond = alpha >= 0
    CS = CS[pistiveAlphaCond, :]

    if normalization:
        # here I implemented the normalization but did not find it useful - the results' improvement is not impressive.
        normalizingFactor = np.fft.fft((w**2) * D, int(R * (1 + (x.size - Nw) / R)))[:np.sum(pistiveAlphaCond)]
        normalizingFactor *= fs * X_w_d.shape[1]
        CS = (CS.T / normalizingFactor).T
        normalizingFactor_abs = np.abs(normalizingFactor)
        normalizingFactorCond = normalizingFactor_abs / np.max(normalizingFactor_abs) > 0.95
        CS = CS[normalizingFactorCond, :]
    else:
        normalizingFactorCond = np.ones(np.sum(pistiveAlphaCond), dtype=bool)


    # Cyclic Coherence
    CS_abs = np.abs(CS)
    if approxCoh:
        CCoh = CS_abs / CS_abs[0, :]
    else:
        inds = np.atleast_2d(np.arange(f.size)) - np.atleast_2d((np.arange(CS.shape[0]) * Nw) / (R * alpha.size)).T
        inds = inds.astype(int)
        CCoh = CS_abs / np.sqrt(CS_abs[0, :] * CS_abs[0, inds])
        CS = CS[:, f >= fMin]
        CCoh = CCoh[:, f >= fMin]

    alpha = alpha[pistiveAlphaCond][normalizingFactorCond]
    f = f[f >= fMin]

    return CS, CCoh, f, alpha

In [ ]:
def getPvalCCoh(x, fs, alphaMax=2000, dfreq=500, alphaLims=[0.019, 0.501]):
    CS, CCoh, f, alpha = getCS(x, fs, alphaMax=alphaMax, dfreq=dfreq, normalization=False)
    CCoh_abs = np.abs(CCoh.T)
    edgeCond = (alpha > alphaMax * alphaLims[0]) & (alpha < alphaMax * alphaLims[1])
    CCoh_abs = CCoh_abs[:, edgeCond]
    alpha = alpha[edgeCond]
    EES = CCoh_abs.sum(axis=0) 
    
    cond100 = np.mod(alpha, 100) == 0
    EES100 = EES[cond100]
#     standardized Euclidean distance
    SED = seuclidean(
        EES100, 
        EES[~cond100].mean() * np.ones_like(EES100), 
        EES[~cond100].var() * np.ones_like(EES100)
    )
    pValue = chi2(EES100.size).sf(SED)
    
    return pValue, CCoh_abs, alpha, f, EES

In [ ]:
alphaMax = 2000
dfreq = 1500
fs = 1000 / df.iloc[1, 0] 

In [ ]:
for ch in np.arange(1, df.shape[1]):
    x = df.iloc[:, ch]
    pValue, CCoh_abs, alpha, f, EES = getPvalCCoh(df.iloc[:, ch].values, fs, alphaMax=alphaMax, dfreq=dfreq)

    plt.figure(figsize=figsize)
    plt.pcolormesh(alpha, f, CCoh_abs, shading='auto', vmax=np.percentile(CCoh_abs, 99.8))
    plt.xlabel('Modulation frequency [Hz]')
    plt.ylabel('Carrier Frequency [Hz]')
    plt.title(file.split('/')[-1][:-5] + ': Spectal cyclic coherence; channel ' + str(ch))
    plt.xticks(np.arange(50, alpha[-1], 50))
    plt.colorbar()
    plt.show()
    
    plt.figure(figsize=figsize)
    plt.plot(alpha, EES)
    plt.xlabel('Modulation frequency [Hz]')
    plt.title(file.split('/')[-1][:-5]  + ': channel ' + str(ch) + '; p-value={:.3f}'.format(pValue))
    plt.xticks(np.arange(50, alpha[-1], 50))
    plt.ylabel('EES')
    plt.grid()
    plt.show()